In [1]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.7.0


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


2022-03-01 13:19:29.278062: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2844 - sparse_categorical_accuracy: 0.6172
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 16ms/step - loss: 1.2127 - sparse_categorical_accuracy: 0.6360 - val_loss: 0.7426 - val_sparse_categorical_accuracy: 0.7810
Epoch 2/10
22/32 [===================>..........] - ETA: 0s - loss: 0.4467 - sparse_categorical_accuracy: 0.8835
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - loss: 0.4348 - sparse_categorical_accuracy: 0.8780 - val_loss: 0.5511 - val_sparse_categorical_accuracy: 0.8290
Epoch 3/10
25/32 [======================>.......] - ETA: 0s - loss: 0.2966 - sparse_categorical_accuracy: 0.9225
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.2950 - sparse_categorical_accuracy: 0.9220 - val_loss: 0.4773 - val_sparse_categorical_accuracy: 0.8580
Epoch 4/10
32/

In [6]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

In [7]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3005 - sparse_categorical_accuracy: 0.1430 - 169ms/epoch - 5ms/step
Untrained model, accuracy: 14.30%


In [8]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4196 - sparse_categorical_accuracy: 0.8660 - 74ms/epoch - 2ms/step
Restored model, accuracy: 86.60%


In [9]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [10]:
os.listdir(checkpoint_dir)

['cp-0005.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index']

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4963 - sparse_categorical_accuracy: 0.8720 - 188ms/epoch - 6ms/step
Restored model, accuracy: 87.20%


In [13]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4963 - sparse_categorical_accuracy: 0.8720 - 198ms/epoch - 6ms/step
Restored model, accuracy: 87.20%


In [14]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 1s 4ms/step - loss: 1.1685 - sparse_categorical_accuracy: 0.6520
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4295 - sparse_categorical_accuracy: 0.8810
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2951 - sparse_categorical_accuracy: 0.9190
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2035 - sparse_categorical_accuracy: 0.9570
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1539 - sparse_categorical_accuracy: 0.9670


2022-03-01 13:20:07.279684: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [15]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets            keras_metadata.pb saved_model.pb    variables


In [16]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4206 - sparse_categorical_accuracy: 0.8600 - 200ms/epoch - 6ms/step
Restored model, accuracy: 86.00%
(1000, 10)


In [18]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.2033 - sparse_categorical_accuracy: 0.6540
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4489 - sparse_categorical_accuracy: 0.8810
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2985 - sparse_categorical_accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2101 - sparse_categorical_accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.1688 - sparse_categorical_accuracy: 0.9650


In [19]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [20]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4259 - sparse_categorical_accuracy: 0.8650 - 226ms/epoch - 7ms/step
Restored model, accuracy: 86.50%
